# CH6. Django의 웹 서버 연동 원리

## 1. 장고의 wsgi.py

: 프로젝트를 처음 생성할 때 함께 생성되는 파일로, WSGI 규격에 따라 호출 가능한 애플리케이션 객체를 정의함

```
# 정의되어있는 객체
application = get_wsgi_application()
```
**객체명은 반드시 application이여야 함**

> application 객체 지정 위치   
> * Apache : httd.conf   
> * NGINX/uWSGI : httd.conf 설정파일의 WSGIScriptAlias 지시자 또는 uwsgi.ini 설정 파일의 module   
> * runserver : settings 모듈의 WSGI_APPLICATION   
> ```
> import os   
> os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mysite.settings")   
> (혹은) python manage.py runserver --settings = mysite.settings
> ```

## 2. 장고의 WSGI 인터페이스

: 운영 환경에서는 NGINX와 같은 웹 서버 프로그램이 클라이언트 요청을 수신하므로, 요청을 수신한 웹 서버가 uWSGI와 같은 WAS 서버를 통하여 장고 웹 애플리케이션을 호출할 수 있어야 함

> **애플리케이션 서버 종류**   
> * 웹 서버 : Apache httpd 또는 nignx와 같은 웹 서버 프로그램   
> * 웹 애플리케이션 서버 : uwsgi, gunicom과 같은 WAS 서버 프로그램   
> * 애플리케이션 : 장고 프레임워크를 사용해, 우리가 개발하는 파이썬 프로그램   
> * 장고 : 파이썬 웹 애플리케이션을 만들어주는 프레임워크

* 장고에서 만든 프로그램은 WAS 서버에서 호출될 수 있도록 WSGI 규격을 준수해야함   
자동 생성된 wsgi.py 파일에서 역할을 수행
```
mysite/wsgi.py
application = get_wsgi_appplication()
def get_wsgi_application() :
    return WSGIHandler()
```

* 웹 서버 또는 WAS 서버가 장고 애플리케이션을 실행하기 위해서는 application 호출자가 정의된 wsgi.py 파일의 위치를 알아야 함   
  **웹/WAS 서버의 설정 파일에 application 호출자의 경로를 정의해야 함**
```
(runserver)
mysite/settings.py
WSGI_APPLICATION = 'mysite.wsgi.aplication'
```

## 3. 운영 서버 적용 전 장고의 설정 변경 사항

: 아파치 등과 같은 웹 서버와 연동하는 단계는 장고의 애플리케이션 개발이 완료되고, 개발용 웹 서버인 runserver에서 정상적으로 동작하는 것을 확인한 이후 시점이므로, 운영 서버에 적용하기 위해서는 보안, 성능 등을 고려하여 운영 환경에 맞는 설정으로 변경해야 할 사항들이 있다.

* 필요 항목 설정 체크   
```
python manage.py check --deploy
```

1. SECRET_KEY   
: 암호화 시 사용하는 항목으로, 외부에 노출되어서는 안됨   
```
mysite/settings.py
import os
SECRET_KEY = os.environ['SECRET_KEY']
```

2. DEBUG   
: 디버그 정보가 노출되지 않도록 설정
```
mysite/settings.py
DEBUG = FALSE
# CSRF 공격 방지
ALLOWED_HOSTS = ['localhost', '127.0.0.1']
```

3. 정적 파일의 위치
: 아파치와 같은 웹 서버에게 정적 파일의 위치를 알려주어야 함
```
mysite/settings.py
STATIC_ROOT = os.path.join(BASE_DIR, 'www_dir', 'static')
```
  **웹 서버의 설정 파일에도 등록해주어야 함**
  
4. 정적 파일 명령
: settings 모듈의 STATICFILES_DIRTS 항목에 STATIC_ROOT에서 정의된 디렉토리가 포함되면 안된다는 명령
```
python manage.py collectstatic
```

5. 접근 권한 설정

: 개발 모드에서 runserver을 실행시킨 사용자의 권한으로 데이터베이스 파일이나 로그 파일을 액세스 하는 것과 달리, 운영 모드에서는 웹 서버 프로세스의 소유자 (apache) 권한으로 해당 파일들을 액세스 할 수 있어야 함
```
# 본래 설정
mysite/settings.py
DATABASES = {
        . . .
        'NAME' : os.path.join(BASE_DIR, 'db', 'db.sqlite3'),
}
```

```
# 변경 설정
# 리눅스 명령
# DB 변경
cd /home/hgm/study/ch8
mkdir db
mv db.sqlite3 db/
chmode 777 db/
chmod 666 db/db.sqlite3

# 권한 변경
cd /home/hgm/study/ch8
chmode 777 logs/
chmode 666 logs/mysite.log
```

6. 캐시 서버, 데이터베이스 서버   
: 보안이 취약하므로 외부에서 직접 액세스 하지 않게 웹 서버 또는 WAS 서버로 연결 제한   


7. 발신자 주소 변경   
: 메일 기능이 있는 애플리케이션에 한하여 보안을 향상시키기 위해 **SERVER_EMAIL** 및 **DEFAULT_FROM_EMAIL** 설정 항목을 이용하여 변경